In [1]:
import scripts
import tensorflow as tf
import importlib

2023-04-19 16:58:54.688882: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
importlib.reload(scripts)

<module 'scripts' from '/Users/alfred/Documents/chinese_mnist_analysis/scripts.py'>

# Solving Chinese MNIST using Transfer Learning

## Some Hyperparameters

In [4]:
NUMBER_OF_CLASSES = 15
IMAGE_SIZE = (64, 64)
IMAGE_SHAPE = IMAGE_SIZE + (3,)

CLASSIFIER_LEARNING_RATE = 0.0001
CLASSIFIER_NUMBER_OF_EPOCHS = 1
BATCH_SIZE = 1
OPTIMIZER = tf.keras.optimizers.Adam
LOSS_MEASURE = tf.keras.losses.SparseCategoricalCrossentropy

## Dataloaders

In [5]:
small_training_dataloader = tf.keras.utils.image_dataset_from_directory("./debug_dataset/",
                                                                        batch_size=BATCH_SIZE,
                                                                        image_size=IMAGE_SIZE,
                                                                        seed=413)
medium_training_dataloader = tf.keras.utils.image_dataset_from_directory("./debug_dataset/",
                                                                         batch_size=BATCH_SIZE,
                                                                         image_size=IMAGE_SIZE,
                                                                         seed=413)
large_training_dataloader = tf.keras.utils.image_dataset_from_directory("./debug_dataset/",
                                                                        batch_size=BATCH_SIZE,
                                                                        image_size=IMAGE_SIZE,
                                                                        seed=413)
validation_dataloader = tf.keras.utils.image_dataset_from_directory("./debug_dataset/",
                                                                    batch_size=BATCH_SIZE,
                                                                    image_size=IMAGE_SIZE,
                                                                    seed=413)

Found 15 files belonging to 15 classes.


2023-04-19 17:14:53.713542: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 15 files belonging to 15 classes.
Found 15 files belonging to 15 classes.
Found 15 files belonging to 15 classes.


### Take a Peek

In [6]:
# scripts.peek_into_dataloader(debug_dataloader)

### Cache and Prefetch

In [7]:
small_training_dataloader = small_training_dataloader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
medium_training_dataloader = medium_training_dataloader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
large_training_dataloader = large_training_dataloader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataloader = validation_dataloader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

### Pre-Trained Base Models
The models expect inputs to be in the range [0, 255].

In [8]:
pretrained_efficientnet_to_fine_tune_with_small_dataset = tf.keras.applications.EfficientNetV2S(include_top=False, input_shape=IMAGE_SHAPE) 
pretrained_efficientnet_to_fine_tune_with_medium_dataset = tf.keras.applications.EfficientNetV2S(include_top=False, input_shape=IMAGE_SHAPE)
pretrained_efficientnet_to_fine_tune_with_large_dataset = tf.keras.applications.EfficientNetV2S(include_top=False, input_shape=IMAGE_SHAPE)

### Full Models
Adding our own classifiers.

In [9]:
pretrained_efficientnet_to_fine_tune_with_small_dataset = scripts.add_classifier(pretrained_efficientnet_to_fine_tune_with_small_dataset, IMAGE_SHAPE, NUMBER_OF_CLASSES)
pretrained_efficientnet_to_fine_tune_with_medium_dataset = scripts.add_classifier(pretrained_efficientnet_to_fine_tune_with_medium_dataset, IMAGE_SHAPE, NUMBER_OF_CLASSES)
pretrained_efficientnet_to_fine_tune_with_large_dataset = scripts.add_classifier(pretrained_efficientnet_to_fine_tune_with_large_dataset, IMAGE_SHAPE, NUMBER_OF_CLASSES)

### Inspection

In [10]:
print("Number of layers in the efficientnet base model: {}".format(len(pretrained_efficientnet_to_fine_tune_with_small_dataset.layers[1].layers)))

Number of layers in the efficientnet base model: 513


In [11]:
pretrained_efficientnet_to_fine_tune_with_small_dataset.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 efficientnetv2-s (Functiona  (None, 2, 2, 1280)       20331360  
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 5120)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              5243904   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 15)                15375     
                                                             

## Training the Classifiers

In [12]:
history_of_training_classifier_with_small_dataset = scripts.train_classifier(pretrained_efficientnet_to_fine_tune_with_small_dataset, 
                                                                             small_training_dataloader,
                                                                             validation_dataloader,
                                                                             CLASSIFIER_NUMBER_OF_EPOCHS,
                                                                             OPTIMIZER,
                                                                             CLASSIFIER_LEARNING_RATE,
                                                                             LOSS_MEASURE,
                                                                             True,
                                                                             ["accuracy"])

Number of trainable parameters before freezing the base model: 25436767
Number of trainable parameters after freezing the base model: 5259279
15/15 [==============================] - 26s 480ms/step - loss: 3.2190 - accuracy: 0.0667 - val_loss: 2.0658 - val_accuracy: 0.5333


In [13]:
history_of_training_classifier_with_medium_dataset = scripts.train_classifier(pretrained_efficientnet_to_fine_tune_with_medium_dataset, 
                                                                              medium_training_dataloader,
                                                                              validation_dataloader,
                                                                              CLASSIFIER_NUMBER_OF_EPOCHS,
                                                                              OPTIMIZER,
                                                                              CLASSIFIER_LEARNING_RATE,
                                                                              LOSS_MEASURE,
                                                                              True,
                                                                              ["accuracy"])

Number of trainable parameters before freezing the base model: 25436767
Number of trainable parameters after freezing the base model: 5259279
15/15 [==============================] - 26s 517ms/step - loss: 3.4116 - accuracy: 0.0000e+00 - val_loss: 2.1160 - val_accuracy: 0.4667


In [14]:
history_of_training_classifier_with_large_dataset = scripts.train_classifier(pretrained_efficientnet_to_fine_tune_with_large_dataset, 
                                                                             large_training_dataloader,
                                                                             validation_dataloader,
                                                                             CLASSIFIER_NUMBER_OF_EPOCHS,
                                                                             OPTIMIZER,
                                                                             CLASSIFIER_LEARNING_RATE,
                                                                             LOSS_MEASURE,
                                                                             True,
                                                                             ["accuracy"])

Number of trainable parameters before freezing the base model: 25436767
Number of trainable parameters after freezing the base model: 5259279
15/15 [==============================] - 26s 524ms/step - loss: 3.3449 - accuracy: 0.0000e+00 - val_loss: 2.1250 - val_accuracy: 0.5333


## Fine-Tuning the Entire Models

Only fine tune after the classifier have been trained.

### Some Hyperparameters

In [15]:
FINE_TUNING_LEARNING_RATE = CLASSIFIER_LEARNING_RATE / 10
FINE_TUNING_NUMBER_OF_EPOCHS = 1
TOTAL_NUMBER_OF_EPOCHS = CLASSIFIER_NUMBER_OF_EPOCHS + FINE_TUNING_NUMBER_OF_EPOCHS
NUMBER_OF_LAYERS_TO_FREEZE = 100

In [16]:
history_of_fine_tuning_with_small_dataset = scripts.fine_tune(pretrained_efficientnet_to_fine_tune_with_small_dataset,
                                                              NUMBER_OF_LAYERS_TO_FREEZE,
                                                              history_of_training_classifier_with_small_dataset,
                                                              small_training_dataloader,
                                                              validation_dataloader,
                                                              TOTAL_NUMBER_OF_EPOCHS,
                                                              OPTIMIZER,
                                                              FINE_TUNING_LEARNING_RATE,
                                                              LOSS_MEASURE,
                                                              True,
                                                              ["accuracy"])

Number of trainable parameters after unfreezing the entire base model: 25436767
Number of trainable parameters after freezing the first 100 layers of the base model: 24228183
Epoch 1/2
15/15 [==============================] - 66s 689ms/step - loss: 2.1223 - accuracy: 0.4667 - val_loss: 1.7302 - val_accuracy: 0.5333
Epoch 2/2
15/15 [==============================] - 5s 347ms/step - loss: 1.7948 - accuracy: 0.4667 - val_loss: 1.4293 - val_accuracy: 0.7333


In [19]:
history_of_fine_tuning_with_medium_dataset = scripts.fine_tune(pretrained_efficientnet_to_fine_tune_with_medium_dataset,
                                                               NUMBER_OF_LAYERS_TO_FREEZE,
                                                               history_of_training_classifier_with_medium_dataset,
                                                               medium_training_dataloader,
                                                               validation_dataloader,
                                                               TOTAL_NUMBER_OF_EPOCHS,
                                                               OPTIMIZER,
                                                               FINE_TUNING_LEARNING_RATE,
                                                               LOSS_MEASURE,
                                                               True,
                                                               ["accuracy"])

Number of trainable parameters after unfreezing the entire base model: 25436767
Number of trainable parameters after freezing the first 100 layers of the base model: 24228183
Epoch 1/2
15/15 [==============================] - 77s 935ms/step - loss: 1.6883 - accuracy: 0.6000 - val_loss: 1.2311 - val_accuracy: 0.7333
Epoch 2/2
15/15 [==============================] - 7s 476ms/step - loss: 1.2111 - accuracy: 0.8000 - val_loss: 0.9229 - val_accuracy: 1.0000


In [18]:
history_of_fine_tuning_with_large_dataset = scripts.fine_tune(pretrained_efficientnet_to_fine_tune_with_large_dataset,
                                                              NUMBER_OF_LAYERS_TO_FREEZE,
                                                              history_of_training_classifier_with_large_dataset,
                                                              large_training_dataloader,
                                                              validation_dataloader,
                                                              TOTAL_NUMBER_OF_EPOCHS,
                                                              OPTIMIZER,
                                                              FINE_TUNING_LEARNING_RATE,
                                                              LOSS_MEASURE,
                                                              True,
                                                              ["accuracy"])

Number of trainable parameters after unfreezing the entire base model: 25436767
Number of trainable parameters after freezing the first 100 layers of the base model: 24228183
Epoch 1/2
15/15 [==============================] - 89s 994ms/step - loss: 2.1450 - accuracy: 0.4667 - val_loss: 1.8377 - val_accuracy: 0.6000
Epoch 2/2
15/15 [==============================] - 6s 394ms/step - loss: 1.7731 - accuracy: 0.5333 - val_loss: 1.5068 - val_accuracy: 0.7333


## Results